In [1]:
from pyspark.sql import SparkSession 
import pyspark.sql.functions as f
spark = SparkSession.builder.master("local").appName("App").getOrCreate()



Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/05 18:09:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = (spark
          .read
          .format("csv")
          .option("header","true")
          .option("inferSchema","true")
          .option("delimiter",";")
          .load("vacinados.csv"))

### window function, para usar import o pacote

In [3]:
from pyspark.sql.window import Window

## Particionando arquivos e salvando em parquet
### o PartitionBy particiona o arquivo em varios arquivos separando por categoria, no exemplo a baixo o datafreme foi dividido em varios arquivos por sexo dentro de uma pasta chamada por_sexo

obs: quando salvamos or arquivos assim o campo que foi base do partitionby nao é escrito no arquivo, nesse exemplo o sexo. sijestao criar um novo campo se for preciso ter ele escrito

In [5]:
df.write.option("header", True) \
        .partitionBy("sexo") \
        .mode("overwrite") \
        .parquet("em_parquet")

"""
vai salvar em uma pasta coma seguinte estrutura
em_parquet (pasta)
|_sexo=masculino (pasta)
|_sexo=feminino (pasta)
|_sexo=outros (pasta)
"""

'\nvai salvar em uma pasta coma seguinte estrutura\nem_parquet\n|_sexo=masculino\n|_sexo=feminino\n|_sexo=outros\n'

## Lendo Arquivos parquet particionado
### se existir mais de um arquivo em uma pasta basta colocar * que todos arquivos vâo ser lidos
### isso também serve para subpatas o spark é esperto a ponto de entrar  em cada pasta e ler os aruivos

In [12]:
### lendo de tadas as nossas pastas
"""
em_parquet (pasta)
|_sexo=masculino (pasta)
|_sexo=feminino (pasta)
|_sexo=outros (pasta)
"""
dfFull = spark.read.parquet("em_parquet/*")

In [21]:
### lendo de uma pasta especifica
"""
em_parquet (pasta)
|_sexo=masculino (pasta)
|_sexo=feminino (pasta)
|_sexo=outros (pasta)
"""
dfOutros = spark.read.parquet("em_parquet/sexo=OUTROS/*")

In [22]:
dfOutros.show()

+--------------+--------------------+--------------------+--------------------+------+----+--------------+--------------------+
|            pf|                nome|               grupo|              vacina|  lote|dose|data_vacinacao|     local_vacinacao|
+--------------+--------------------+--------------------+--------------------+------+----+--------------+--------------------+
|***.302.844-**|ABDIAS DE MELO TE...|PESSOAS COM COMOR...|3 - COMIRNATY (PF...|EX2405|   1|    2021-05-13| DRIVE THRU GERALDÃO|
|***.302.844-**|ABDIAS DE MELO TE...|PESSOAS COM COMOR...|3 - COMIRNATY (PF...|FD7221|   2|    2021-08-06|CENTRO DE VACINAÇ...|
+--------------+--------------------+--------------------+--------------------+------+----+--------------+--------------------+



In [24]:
dfOutros.printSchema()

root
 |-- pf: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- grupo: string (nullable = true)
 |-- vacina: string (nullable = true)
 |-- lote: string (nullable = true)
 |-- dose: integer (nullable = true)
 |-- data_vacinacao: string (nullable = true)
 |-- local_vacinacao: string (nullable = true)

